In [1]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 13.0MB/s 
     |████████████████████████████████| 133kB 33.3MB/s 
     |████████████████████████████████| 7.5MB 32.9MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
ERROR: botocore 1.20.69 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 33kB/s 
     |████████████████████████████████| 3.8MB 34.7MB/s 
     |████████████████████████████████| 512kB 38.8MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=5b9ef1d7b768e742853f2c7abeab2472bd82a85324b5b1d6c9be9ffd7ca0b2ba
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing instal

In [3]:
!pip install bertviz

     |████████████████████████████████| 153kB 29.3MB/s 
     |████████████████████████████████| 2.1MB 32.5MB/s 
     |████████████████████████████████| 1.2MB 31.5MB/s 
     |████████████████████████████████| 3.3MB 30.7MB/s 
     |████████████████████████████████| 901kB 34.7MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import joblib
tokenized_text = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/agritrend_tokenized.joblib")

In [8]:
from bertviz import head_view
from transformers import BertTokenizer, BertModel

In [ ]:
# Load model and retrieve attention
model_version = 'm-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0'
do_lower_case = True
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)


In [20]:
import numpy as np

In [ ]:
filea = open('attentions.csv',"w")
filet = open('tokens.csv',"w")
for k in tokenized_text:
  oring_to = np.array(k)
  sentence_a =""
  i = 0
  for kk in k:
    if i == 0:
      sentence_a = kk
    else:
      sentence_a = str(sentence_a) +" "+str(kk)
    i = i+1
  sentence_b = ""
  inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
  token_type_ids = inputs['token_type_ids']
  input_ids = inputs['input_ids']
  attention = model(input_ids, token_type_ids=token_type_ids)[-1]

  attentions_values = []
  for i in range(0,12):
    heads_values = []
    deatt = attention[i].detach().numpy()
    for j in range(0,12):
      matrix = np.array(deatt[0][j])
      vett = matrix.sum(axis=0)
      heads_values.append(vett)
    heads_values = np.array(heads_values)
    head = heads_values.sum(axis=0)
    attentions_values.append(head)
  attentions_values = np.array(attentions_values)
  atts = attentions_values.sum(axis=0)
  #print(atts)
  tokens = oring_to
  tag = False
  tag_score = 0
  tag_word = ''
  ll = 0
  new_att = []
  new_tags=[]
  coun = 0
  for x in tokens:
    if x != '[CLS]' and x !='[SEP]':
      if tag:
        if x == '>':
          tag = False
          coun = coun +1
          tag_score = (tag_score + atts[ll])/ coun
          tag_word = tag_word+x
          new_tags.append(tag_word)
          new_att.append(tag_score)
          tag_score = 0
          tag_word = ''
          coun = 0
        else:
          if tag_word == '<' and x[0] not in ['u','h','n','e','p','t','d']:
            tag = False
            new_tags.append(tag_word)
            new_att.append(tag_score)
            tag_score = 0
            tag_word = ''
            coun = 0
            new_tags.append(x)
            new_att.append(atts[ll])
            coun = coun +1
          else:
            coun = coun +1
            tag_score += atts[ll]
            tag_word += x.replace('##','')
      else:
        if x == '<':
            tag = True
            tag_score = tag_score + atts[ll]
            tag_word = tag_word+x
            coun = coun +1
        else:
            new_att.append(atts[ll])
            new_tags.append(x)
      ll = ll +1

  #new_att = np.array(new_att)
  new_att = np.array([ x /max(new_att) for x in new_att])
  new_tags = np.array(new_tags)
  #print(new_att)
  #print(new_tags)
  #print(np.array(oring_to).shape)
  pp = 0
  if len(new_att) == len(oring_to):
    atts = ''
    tos = ''
    for pp in range (0,len(new_att)):
      atts= atts+str(new_att[pp])+","
      tos = tos + str(new_tags[pp]+",")
    atts = atts[:-1]+"\n"
    tos = tos[:-1]+"\n"
    filea.write(atts)
    filea.flush()
    filet.write(tos)
    filet.flush()
  else:
    print(len(new_att))
    print(oring_to)
    print(new_tags)
filea.close()
filet.close()

  #input_id_list = input_ids[0].tolist() # Batch index 0
  #tokens = tokenizer.convert_ids_to_tokens(input_id_list)
